In [1]:
def get_n_specific_labels(v, sr, n):
    return sr[v][:n].index.values

sr_labels = sr_labels.sort_values()
#get_most_specific_labels(df.en_tags.iloc[100], sr, 1)
%time df.en_tags = df.en_tags.apply(lambda x: get_n_specific_labels(x, sr_labels, 3))

NameError: name 'sr_labels' is not defined

In [8]:
import pandas as pd
import numpy as np
from fasttext import *
%matplotlib inline
import sys
sys.path.append("/Users/evelyn.trautmann/repos/pi-datascience-adhoc-analysis/notebooks/templates/")
import evaluation_metrics

from sklearn.model_selection import train_test_split

import scipy.spatial.distance as sd
import sklearn.metrics.pairwise as pw
import re

In [2]:
#load data
df = pd.read_csv("data/foodcategories_3labels.csv", sep = "\x01")
df.en_tags = df.en_tags.apply(eval)
df = df[df.en_tags.str.len()>0]

In [3]:
label, freq = np.unique(np.concatenate(df.en_tags.values), return_counts = True)
sr_labels = pd.Series(index=label, data=freq).sort_values()


In [4]:
df.en_tags.str.len().value_counts()

3    256401
2     47258
1     25644
Name: en_tags, dtype: int64

# Prepare

In [5]:
#generate features
feature_cols = ['product_name', 'generic_name', 'brands']
print(df[df.en_tags.str.len()==0])
X = df[feature_cols].fillna("").apply(lambda x: " ".join(x), axis = 1)
y = df.en_tags

Empty DataFrame
Columns: [product_name, generic_name, brands, categories, categories_tags, origins, manufacturing_places, labels, emb_codes, countries, main_category, en_tags]
Index: []


In [9]:
# preprocess and split train and test date
X = X.str.lower().apply(lambda x: re.sub(r'[^\w\s]','',x))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

dftrain = y_train.apply(lambda x: " ".join(["__label__" + y
                    for y in x])) + " " + X_train


In [10]:
dftrain_emb = y_train.apply(lambda x: " ".join(x)) + " " + X_train
dftrain_emb.to_csv(
    "train_emb.csv", index = False, sep=";")

In [11]:
dftest = y_test.apply(lambda x: " ".join(["__label__" + y
                    for y in x])) + " " + X_test


In [12]:
dftrain.to_csv(
    "train.csv", index = False, sep=";")
dftest.to_csv(
    "test.csv", index = False, sep=";")

# Train

Uninstalling fasttext-0.8.22:
  Would remove:
    /usr/local/anaconda3/lib/python3.7/site-packages/fastText/*
    /usr/local/anaconda3/lib/python3.7/site-packages/fasttext-0.8.22.dist-info/*
    /usr/local/anaconda3/lib/python3.7/site-packages/fasttext_pybind.cpython-37m-darwin.so
Proceed (y/n)? ^C
Operation cancelled by user


In [13]:
train_supervised("/Users/evelyn.trautmann/projects/openfood/train.csv",
                autotuneValidationFile = "test.csv")

In [17]:
!/Users/evelyn.trautmann/repos/fasttext3/fastText/fasttext supervised \
    -input "/Users/evelyn.trautmann/projects/openfood/train.csv" -output model \
    -autotune-validation "test.csv" -autotune-duration 1000

Progress: 100.0% Trials:   19 Best score:  0.474345 ETA:   0h 0m 0sss27sh 9m 2sm57sh 7m46s18s25s 9s33s Best score:  0.472712 ETA:   0h 5m23sssm28s 3m 6sm25ss50s  0h 0m 2s
Training again with best arguments
Read 2M words
Number of words:  79212
Number of labels: 137
Progress: 100.0% words/sec/thread:  150404 lr:  0.000000 avg.loss:  1.298324 ETA:   0h 0m 0s54s0h 1m56ssm51s44s42s  0h 1m41ss6sm 4sm 1s 0m58ss  0h 0m46ss35s  0h 0m35s 0m33sh 0m24s  0h 0m24sm20sm14s13s 3s2s


In [33]:
train_supervised??

In [18]:
!/Users/evelyn.trautmann/repos/fasttext3/fastText/fasttext dump model.bin args

dim 109
ws 5
epoch 100
minCount 1
neg 5
wordNgrams 4
loss softmax
model sup
bucket 1120579
minn 0
maxn 0
lrUpdateRate 100
t 0.0001


In [19]:
!/Users/evelyn.trautmann/repos/fasttext3/fastText/fasttext test model.bin test.csv 3

N	108670
P@3	0.763
R@3	0.848


In [21]:
model = load_model("model.bin")

In [37]:
model_emb = train_unsupervised("/Users/evelyn.trautmann/projects/openfood/train_emb.csv")

In [ ]:
# model = train_supervised(input="/Users/evelyn.trautmann/projects/openfood/train.csv", 
#                          autotuneValidationFile="test.csv")

# Test

In [22]:
model.test("test.csv", k=3)

(108670, 0.7626269132848685, 0.8481321402864813)

In [23]:
df_test = X_test.to_frame()
df_test.columns = ["feature"]

In [24]:
# determine number of labels
df_test = df_test.join(y_test)

K = int(df_test.en_tags.str.len().quantile(0.75))
K

3

# Multilabel Evaluation 

In [25]:
df_test["prediction"] = df_test.feature.apply(lambda x: model.predict(x, k=K))

for k in range(K):
    df_test["label_predicted%i" %k] = df_test.prediction.str[0].str[k].str.replace("__label__","")
    df_test["confidence%i" %k] = df_test.prediction.str[1].str[k]
    df_test["truth%i" %k] = df_test.en_tags.str[k].str.replace("__label__","")


In [26]:
df_test.count()

feature             108670
en_tags             108670
prediction          108670
label_predicted0    108670
confidence0         108670
truth0              108670
label_predicted1    108670
confidence1         108670
truth1              100092
label_predicted2    108670
confidence2         108670
truth2               84382
dtype: int64

In [27]:
# assertion no empty tags
assert(len(df_test.en_tags[df_test.en_tags.str.len()==0])==0)

Let \{$T_1^{(1)}, ...,T_M^{(1)}$\},...,\{$T_1^{(N)}, ...,T_M^{(N)}$\} be our ground truth, 
\{$P_1^{(1)}, ...,P_M^{(1)}$\},...,\{$P_1^{(N)}, ...,P_M^{(N)}$\} the prediction.

on diagonal we count the events of truth matching prediction

$\sum_{i=1}^N |\{T_1^{(i)}, ...,T_M^{(i)}\}\cap \{P_1^{(i)}, ...,P_M^{(i)}\}|$

so for each diagonal entry we count over all data points (N) and all predictions (M) the total amount when truth is matching prediction.

$C_{kk} =\sum_{i=1}^N \sum_{m=1}^M \chi{1}\{T_m^{(i)}=k,P_m^{(i)}=k\} $

in contrast on off-diagonal we count all events where truth was not covered by prediction.
For the off-diagonal we take only the difference sets into account. If truth for data point i was for example {fish, curry, rice-dish} and prediction was {fish, curry, soup }, we only count 
$T^i$ = {rice_dish} and $P^i$ = {soup} for the off-daigonal. Formally we do the following transformation:

$\{\tilde{T}_m^{(i)}\}_{q=1,..M1} = \{T_m^{(i)}\}_{m=1,..M}\setminus \{P_m^{(i)}\}_{m=1,..M}$ 

$\{\tilde{P}_m^{(i)}\}_{p=1,..M2} = \{P_m^{(i)}\}_{m=1,..M}\setminus \{T_m^{(i)}\}_{m=1,..M}$

Eventually we count all events of the ground truth that were not captured in predictions, which results in

$C_{kl} =\sum_{i=1}^N \sum_{m1=1}^{M1} \sum_{m2=1}^{M2}
\frac{1}{M1}\chi{1}\{\tilde{T}_{m1}^{(i)}=k,\tilde{P}_{m2}^{(i)}=l\} $

Once we have computed a multi-class - multi-label confusion matrix the classification report is straight forward.

Precision for class k is computed as

$Pr_k=\frac{C_{kk}}{\sum_j C_{kj}}$

and Recall

$R_k=\frac{C_{kk}}{\sum_j C_{jk}}$



In [29]:
classes = sr_labels.index.values
#classes = list(set(np.concatenate(df_test.en_tags[df_test.en_tags.str.len()>0].values)))

truth = list()
predictions = list()
for k in range(int(K)):
    truth.append("truth%i" %k)
    predictions.append("label_predicted%i" %k)

In [30]:
%time confusion = evaluation_metrics.get_confusion(df_test, classes, K, truth, predictions)

CPU times: user 7min 9s, sys: 8.43 s, total: 7min 17s
Wall time: 7min 14s


In [31]:
print(np.diag(confusion).sum())
print(confusion.sum().sum())
print(df_test.shape[0])

print(np.diag(confusion).sum()/confusion.sum().sum())

248624.0
293144.0
108670
0.8481292470594657


In [33]:
df_report = evaluation_metrics.get_report(confusion)

In [35]:
evaluation_metrics.get_summary(df_report, confusion)

,precision,recall,f1-score,support
micro avg,0.848129,0.848129,0.848129,293144.0
macro avg,0.715294,0.789017,0.750349,293144.0
weighted avg,0.858766,0.848129,0.853414,293144.0


# Similarity

In [42]:
class_vecs = pd.Series({cl:model_emb.get_sentence_vector(cl) for cl in classes})

In [ ]:
model_emb.get_sentence_vector??

In [69]:
Dst = class_vecs.apply(lambda x: class_vecs.apply(lambda y: sd.euclidean(x,y)))

In [70]:
Dst[mask].count().value_counts()

0    163
dtype: int64

In [75]:
mask = Dst > 0.5
Dst[mask]=np.nan
Dst.apply(np.argmax)

rillettes                                                    rillettes
olives                                                          olives
apple-compotes                                                compotes
frozen-plant-based-foods                      frozen-plant-based-foods
rices                                                            rices
baby-foods                                                  baby-foods
potato-crisps                                            potato-crisps
smoked-fishes                                            smoked-fishes
salmons                                                        salmons
animal-fats                                                animal-fats
fish-and-meat-and-eggs                          fish-and-meat-and-eggs
pasteurized-cheeses                                    labeled-cheeses
crackers                                                      crackers
refrigerated-foods                                  refrigerated-foods
tomato

In [84]:
S = np.exp(-Dst**2).fillna(0)

In [91]:
confusion.sum()

rillettes                              309.666667
olives                                 548.166667
apple-compotes                           0.000000
frozen-plant-based-foods               428.166667
rices                                    0.000000
baby-foods                             517.333333
potato-crisps                          581.333333
smoked-fishes                          644.833333
salmons                                160.666667
animal-fats                            579.500000
fish-and-meat-and-eggs                 634.666667
pasteurized-cheeses                     63.166667
crackers                                 0.000000
refrigerated-foods                     275.000000
tomatoes-and-tomato-products             0.000000
dry-pasta                              563.000000
chicken-breasts                          1.333333
dietary-supplements                    639.000000
tunas                                  523.166667
prepared-vegetables                    602.333333


In [93]:
df_report = evaluation_metrics.get_report(confusion, S)
#.dropna()

In [95]:
evaluation_metrics.get_summary(df_report, confusion, S)

,precision,recall,f1-score,support
micro avg,0.851939,0.851939,0.851939,293144.0
macro avg,0.724225,0.797972,0.759312,293144.0
weighted avg,0.861889,0.851939,0.856885,293144.0


In [752]:
for pair in [("beverages","pork"), ("beef","pork"), ("beverages","biscuits"), ("tunas", "meals-with-fish")]:
    print("{}-{}:{}".format(pair[0], pair[1],
                    sd.euclidean(model_emb.get_sentence_vector(pair[0]),model_emb.get_sentence_vector(pair[1]))))

beverages-pork:1.2664862871170044
beef-pork:0.9527662396430969
beverages-biscuits:1.2621071338653564
tunas-meals-with-fish:0.8387037515640259


In [210]:
pair = pd.Series(classes).sample(2).values
print("{}\t{}:{}".format(pair[0], pair[1],
                sd.euclidean(model_emb.get_sentence_vector(pair[0]),model_emb.get_sentence_vector(pair[1]))))
print("{}\t{}:{}".format(pair[0], pair[1],
                sd.cosine(model_emb.get_sentence_vector(pair[0]),model_emb.get_sentence_vector(pair[1]))))

dried-plant-based-foods	desserts:1.1720834970474243
dried-plant-based-foods	desserts:0.6868899166584015


In [140]:
for c1 in classes:
    for c2 in classes:
        gamma = 1.0
        var = 1.0
        dist_cos = sd.cosine(model_emb.get_sentence_vector(c1),model_emb.get_sentence_vector(c2))
        dist = sd.euclidean(model_emb.get_sentence_vector(c1),model_emb.get_sentence_vector(c2))
        dist_rbf = var * np.exp(-gamma *dist)
        if dist_cos  < .4:
            print("{}\t{}:\t{}".format(c1, c2, dist_cos))

mustards	mustards:	0.0
mustards	condiments:	0.2867031693458557
mustards	sauces:	0.32911592721939087
nuts	nuts:	0.0
nuts	nuts-and-their-products:	0.15761154890060425
vegetables-based-foods	vegetables-based-foods:	0.0
vegetables-based-foods	prepared-vegetables:	0.3953184485435486
vegetables-based-foods	fresh-plant-based-foods:	0.18641942739486694
vegetables-based-foods	plant-based-foods:	0.35364454984664917
vegetables-based-foods	frozen-plant-based-foods:	0.21933895349502563
vegetables-based-foods	fruits-and-vegetables-based-foods:	0.15080010890960693
vegetables-based-foods	canned-plant-based-foods:	0.35071080923080444
vegetables-based-foods	fruits-based-foods:	0.3221844434738159
vegetables-based-foods	canned-vegetables:	0.31166863441467285
frozen-foods	frozen-foods:	0.0
frozen-foods	frozen-ready-made-meals:	0.23028844594955444
frozen-foods	frozen-desserts:	0.28185945749282837
frozen-foods	ice-creams-and-sorbets:	0.3977077603340149
frozen-foods	frozen-plant-based-foods:	0.216418206691741

chocolate-candies	candies:	0.2604122757911682
chocolate-candies	chocolate-candies:	0.0
chocolate-candies	bars:	0.3596424460411072
chocolate-candies	milk-chocolates:	0.31962037086486816
chocolate-candies	confectioneries:	0.23213708400726318
chocolate-candies	bonbons:	0.1409364938735962
chocolate-candies	chocolates:	0.34391647577285767
bars	chocolate-candies:	0.3596424460411072
bars	bars:	0.0
bars	dietary-supplements:	0.3989158272743225
milk-chocolates	chocolate-candies:	0.31962037086486816
milk-chocolates	milk-chocolates:	0.0
milk-chocolates	dark-chocolates:	0.14226102828979492
milk-chocolates	chocolates:	0.07278323173522949
fresh-foods	fresh-foods:	0.0
fresh-foods	fresh-plant-based-foods:	0.24951088428497314
fresh-foods	refrigerated-foods:	0.31958574056625366
dried-plant-based-foods	dried-plant-based-foods:	0.0
dried-plant-based-foods	dried-products-to-be-rehydrated:	0.3571828007698059
dried-plant-based-foods	fresh-plant-based-foods:	0.38379961252212524
dried-plant-based-foods	dried-fr

cakes	pastries:	0.32887518405914307
cakes	cakes:	0.0
cakes	biscuits-and-cakes:	0.25092077255249023
farming-products	bee-products:	0.1330583691596985
farming-products	honeys:	0.14338195323944092
farming-products	farming-products:	0.0
fresh-plant-based-foods	vegetables-based-foods:	0.18641942739486694
fresh-plant-based-foods	fresh-foods:	0.24951088428497314
fresh-plant-based-foods	dried-plant-based-foods:	0.38379961252212524
fresh-plant-based-foods	fresh-plant-based-foods:	0.0
fresh-plant-based-foods	plant-based-foods:	0.3309677243232727
fresh-plant-based-foods	frozen-plant-based-foods:	0.23981225490570068
fresh-plant-based-foods	fruits-and-vegetables-based-foods:	0.32991957664489746
fresh-plant-based-foods	canned-plant-based-foods:	0.2682117223739624
fresh-plant-based-foods	canned-vegetables:	0.3768343925476074
extra-virgin-olive-oils	vegetable-fats:	0.39145219326019287
extra-virgin-olive-oils	vegetable-oils:	0.1401221752166748
extra-virgin-olive-oils	extra-virgin-olive-oils:	0.0
extra-

breads	plant-based-foods:	0.36360782384872437
breads	breads:	0.0
breads	cereals-and-potatoes:	0.24324196577072144
dried-fruits	dried-plant-based-foods:	0.0926508903503418
dried-fruits	dried-fruits:	0.0
dried-fruits	dried-products:	0.20455139875411987
sauces	mustards:	0.32911592721939087
sauces	tomato-sauces:	0.29387664794921875
sauces	groceries:	0.18155956268310547
sauces	condiments:	0.37910956144332886
sauces	sauces:	0.0
fruits	fruits:	0.0
fruits	fruits-based-foods:	0.2734556794166565
cooked-pressed-cheeses	labeled-cheeses:	0.1028708815574646
cooked-pressed-cheeses	cheeses:	0.3174574375152588
cooked-pressed-cheeses	aoc-cheeses:	0.11428523063659668
cooked-pressed-cheeses	pasteurized-cheeses:	0.26512742042541504
cooked-pressed-cheeses	french-cheeses:	0.19325828552246094
cooked-pressed-cheeses	cooked-pressed-cheeses:	0.0
cooked-pressed-cheeses	cow-cheeses:	0.188196063041687
olive-tree-products	olives:	0.39100390672683716
olive-tree-products	vegetable-fats:	0.37292593717575073
olive-tree-

chocolates	dark-chocolates:	0.07654464244842529
chocolates	chocolates:	0.0
fruit-yogurts	yogurts:	0.06581997871398926
fruit-yogurts	fermented-milk-products:	0.32905101776123047
fruit-yogurts	fermented-foods:	0.362019419670105
fruit-yogurts	fruit-yogurts:	0.0
hot-beverages	herbal-teas:	0.14151990413665771
hot-beverages	non-alcoholic-beverages:	0.3604074716567993
hot-beverages	sweetened-beverages:	0.3489990234375
hot-beverages	coffees:	0.36793965101242065
hot-beverages	plant-based-beverages:	0.24503523111343384
hot-beverages	teas:	0.14322268962860107
hot-beverages	beverages:	0.3485584855079651
hot-beverages	unsweetened-beverages:	0.33829623460769653
hot-beverages	hot-beverages:	0.0
poultry-meals	microwave-meals:	0.22815024852752686
poultry-meals	frozen-ready-made-meals:	0.3167256712913513
poultry-meals	meals:	0.30896085500717163
poultry-meals	meals-with-meat:	0.10726076364517212
poultry-meals	meat-based-products:	0.21352803707122803
poultry-meals	chickens:	0.37327635288238525
poultry-mea

In [156]:
pd.DataFrame([model_emb.get_sentence_vector(c1),model_emb.get_sentence_vector(c2)]).shape

(2, 100)

In [157]:
pw.rbf_kernel(pd.DataFrame([model_emb.get_sentence_vector(c1),model_emb.get_sentence_vector(c2)]))

array([[1., 1.],
       [1., 1.]])

In [15]:
df.en_tags.value_counts()

[plant-based-foods, fruits-and-vegetables-based-foods, plant-based-foods-and-beverages]    13291
[sauces, groceries]                                                                         9394
[plant-based-foods-and-beverages, juices-and-nectars, beverages]                            7480
[fermented-foods, fermented-milk-products, cheeses]                                         7164
[snacks, biscuits, biscuits-and-cakes]                                                      6781
[plant-based-foods, breads, plant-based-foods-and-beverages]                                6361
[fermented-foods, fermented-milk-products, dairies]                                         6178
[prepared-meats, meats]                                                                     5999
[fermented-milk-products, dairies, cow-cheeses]                                             5928
[breakfasts, plant-based-foods, fruits-and-vegetables-based-foods]                          5250
[plant-based-foods, cereals-an